In [1]:
import pandas as pd
import numpy as np

In [2]:
from itertools import chain
from tqdm.notebook import tqdm

In [12]:
from gensim.models import Word2Vec

In [5]:
train = pd.read_csv('./data/train.csv')

In [6]:
train = train.iloc[:250_000]

In [7]:
train.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [3]:
songs = pd.read_csv('./data/songs.csv')

In [4]:
songs.head()

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [8]:
data = train.merge(songs)

In [9]:
data.head()

,msno,song_id,source_system_tab,source_screen_name,source_type,target,song_length,genre_ids,artist_name,composer,lyricist,language
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
1,e5Ezre9HPuPos+CXQXtmo32E/hHIZTMmo6jG3yRf6UA=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,my library,Local playlist more,local-playlist,1,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
2,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
3,sSexP400TJOZRhx3JB+0s9cqrCnqrlV51B9njoKR1II=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,Online playlist more,online-playlist,0,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0
4,hKdGiUKHVqKkXGHLrc+EzdSW6q0ERAJ2Cs7/L1N0Ae4=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,discover,NaN,online-playlist,0,206471,359,Bastille,Dan Smith| Mark Crew,NaN,52.0


In [42]:
def row_process(row):
    row.fillna('')
    artist = row.artist_name
    composer = row.composer
    lyricist = row.lyricist
    
    artist = artist if artist is not None else ''
    composer = ' '.join(map(lambda x: x.strip(), composer.split('|'))) if composer is not np.nan else ''
    lyricist = ' '.join(map(lambda x: x.strip(), lyricist.split('|'))) if lyricist is not np.nan else ''
    
    return ' '.join([el for el in (artist, composer, lyricist) if el != ''])

In [45]:
song_head = pd.concat([data.msno, data.apply(row_process, axis=1)], axis=1)
song_head.head()

,msno,0
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,Bastille Dan Smith Mark Crew
1,e5Ezre9HPuPos+CXQXtmo32E/hHIZTMmo6jG3yRf6UA=,Bastille Dan Smith Mark Crew
2,pouJqjNRmZOnRNzzMWWkamTKkIGHyvhl/jo4HgbncnM=,Bastille Dan Smith Mark Crew
3,sSexP400TJOZRhx3JB+0s9cqrCnqrlV51B9njoKR1II=,Bastille Dan Smith Mark Crew
4,hKdGiUKHVqKkXGHLrc+EzdSW6q0ERAJ2Cs7/L1N0Ae4=,Bastille Dan Smith Mark Crew


In [50]:
sessions = dict(song_head.groupby("msno")[0].apply(list))
sentences = [values for values in sessions.values() if len(values) > 0]
user_embeddings = Word2Vec(sentences=sentences, vector_size=512, window=5,   
                                 min_count=5, seed=0)

In [65]:
sentences

[['華晨宇 華晨宇 代嶽東/蕭駿峰'],
 ['Maroon 5 Kendrick Lamar Benny Blanco Adam Levine Jacob Kasher Ammar Malik Kurtis McKenzie Jon Mills Alex Ben-Abdullah John Ryan',
  'CHARLIE PUTH',
  'Clean Bandit Jack Patterson Steve Mac Ammar Malik Ina Wroldsen Sean Paul Henriques',
  'Alan Walker Alan Walker Jesper Borgen Anders Froen Gunnar Greve Tommy La Verdi Magnus Bertelsen Iselin Solheim Alan Walker Jesper Borgen Anders Froen Gunnar Greve Tommy La Verdi Magnus Bertelsen Iselin Solheim',
  'twenty one pilots',
  'DJ Snake Justin Bieber William Grigahcine Andrew Watt Ali Tamposi Louis Bell austin roser Lumidee Cedeno Steven Marsden Teddy Mendez Edwin Perez Brian Lee',
  'The Chainsmokers Andrew Taggart Emily Warren Scott Harris Andrew Taggart Emily Warren Scott Harris',
  'The Chainsmokers Andrew Taggart Sara Hejellstrom Nirob Islam Andrew Taggart Sara Hejellstrom Nirob Islam',
  'Justin Bieber Justin Bieber Ed Sheeran Benjamin Joseph Levin',
  'Fifth Harmony Joshua Coleman Jude Demorest Tyrone Griffin 

In [64]:
user_embeddings.wv['Kendrick Lamar']

KeyError: "Key 'Kendrick Lamar' not present"

In [47]:
sentences

[['華晨宇 華晨宇 代嶽東/蕭駿峰'],
 ['Maroon 5 Kendrick Lamar Benny Blanco Adam Levine Jacob Kasher Ammar Malik Kurtis McKenzie Jon Mills Alex Ben-Abdullah John Ryan',
  'CHARLIE PUTH',
  'Clean Bandit Jack Patterson Steve Mac Ammar Malik Ina Wroldsen Sean Paul Henriques',
  'Alan Walker Alan Walker Jesper Borgen Anders Froen Gunnar Greve Tommy La Verdi Magnus Bertelsen Iselin Solheim Alan Walker Jesper Borgen Anders Froen Gunnar Greve Tommy La Verdi Magnus Bertelsen Iselin Solheim',
  'twenty one pilots',
  'DJ Snake Justin Bieber William Grigahcine Andrew Watt Ali Tamposi Louis Bell austin roser Lumidee Cedeno Steven Marsden Teddy Mendez Edwin Perez Brian Lee',
  'The Chainsmokers Andrew Taggart Emily Warren Scott Harris Andrew Taggart Emily Warren Scott Harris',
  'The Chainsmokers Andrew Taggart Sara Hejellstrom Nirob Islam Andrew Taggart Sara Hejellstrom Nirob Islam',
  'Justin Bieber Justin Bieber Ed Sheeran Benjamin Joseph Levin',
  'Fifth Harmony Joshua Coleman Jude Demorest Tyrone Griffin 